# Project 3 Data Collection

The challenge of Project 3 is to collect data from the reddit submissions [API](https://api.pushshift.io/reddit/search/submission), which contains data from various subreddits across reddit. Our goal here is to extract a few thousand rows of data from the API to use in the project, and to export them as a csv. We'll be pulling from the r/Fantasy and r/scifi subreddits for the project. Instructions for how to utilize the API can be found on the API's [Github page](https://github.com/pushshift/api). 

## Loading Packages

In [28]:
import time
import datetime as dt
import requests
import pandas as pd
pd.set_option("display.max_columns", None)

## Extracting Data

According to the instructions on the github page, there are several parameters through which we can specify what we want from the API. For our purposes, we only have a few criteria - we want to specify which subreddit we want to pull from, the amount of entries, and how far back we want to go. It's also important to note that we will not be going through the comments section for this project, only submissions.  

Below is a function that will abstract all the work for extracting our data from the API, while also cleaning some of it for us. We want to get as much data as possible from one consecutive timeframe, so the function will pull from a few days ago, then pull from a few days before that until it finishes iterating. The `day_window` parameter determines how many days to go back, while the `n` parameter is how many times we want to go back that many days. So, if `day_window` is set to 10, and `n` is set to 5, the function will loop through and collect 50 days worth of data. It's important to know that currently, the reddit API only allows for users to extract 100 observations per pull. Given the number of posts per subreddit, we'll have to play around with the function to determine how many days we'll need to space our extractions in order to avoid overlapping data, since we'll be pulling the max observations each time. This function extracts the data as a JSON, and converts it into a dataframe. Since we don't want to be rude and pull all our data at once, the iterations will be spaced out by a period of 2 seconds each so as not to affect the API's server, and in general, to not be rude. The function will also print out whichever set of data it is extracting each time it iterates.  

Each iteration will create an new dataset with 100 rows. We'll concatenate all the datasets into one large dataset that will then be cleaned. Any overlapping data will be cleaned, and unnecessary columns will be dropped. The remaining columns will be: `title`, `selftext`,`subreddit`, `created_utc`, `author`, `num_comments`, `score`, and `is_self`. For more information on these columns, look at the data dictionary in the [README](../README.md). Because the date and time for the data is stored in epoch time, we'll also create a new column that will record the dates in normal mm/dd/yyyy format. For those interested in reproducing this project, the start date for the data is from July 20, 2020. The dataframe will also drop any removed or deleted submissions as those are not useful, as well as any empty values from the submissions. Finally, the dataframe's index is reset, giving us a clean, useable dataframe.

In [166]:
# This code here was copied from Mahdi's intro to project 3 lecture
def query_pushshift(subreddit, kind = 'submission', day_window = 30, n = 5):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem = f"{BASE_URL}?subreddit={subreddit}&size=100" # always pulling max of 100
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    # immediately reset the index so the data frame can tell the difference between the various rows, otherwise, it
    # can drop multiple rows when that was not intended.
    full.reset_index(drop = True, inplace = True)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True, which indicates that the submission is a text submission
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    # drop any removed, deleted, or null texts
    full.drop(index = (full.loc[full["selftext"] == "[removed]",:].index), inplace = True)
    full.drop(index = (full.loc[full["selftext"] == "[deleted]",:].index), inplace = True)
    full.drop(index = (full.loc[full["selftext"].isna(),:].index), inplace = True)
    # reset the index one final time now that all the cleaned data is collected
    full.reset_index(drop = True, inplace = True)
    
    print("Query Complete!")    
    return full 

### The Fantasy Subreddit
After playing around with a few options, we determined that the fantasy subreddit reaches 100 posts every 2 days or so, and our goal was for a dataframe of about 2,500 rows. So, we collected 72 days of data, which meant that from the 3,600 original rows, about 900 rows were dropped due to being duplicates, nontext posts, or removed or deleted posts.

In [177]:
fantasy_df = query_pushshift("fantasy", day_window = 2, n = 36)

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=2d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=4d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=6d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=8d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=10d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=12d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=14d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=16d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&size=100&after=18d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=fantasy&siz

In [189]:
# take a look at our dataset
fantasy_df.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,What are NEWLY published and noticeably popula...,I've asked the same question last year I think...,Fantasy,1595292735,uera,68,37,True,2020-07-20
1,LF Recommendation: Fantasy with long travel ep...,Hi looking for recommendation for a top prefer...,Fantasy,1595295818,Overthrown77,16,5,True,2020-07-20
2,What is the best Fantasy book you've read that...,usually I don't enjoy something below four sta...,Fantasy,1595304562,OraclePreston,55,19,True,2020-07-21
3,Favorite fantasy names?,"I love fantasy, and I also have a thing for na...",Fantasy,1595305376,omnomenclature,56,17,True,2020-07-21
4,Anyone can recommend me a book without having ...,Idk how else to put the title but I really wan...,Fantasy,1595305716,UlyssesCourier,61,33,True,2020-07-21


In [197]:
# Confirm the size of our data
print (f"There are {fantasy_df.shape[0]} rows of data in this dataset.")
print (f"There are {fantasy_df.shape[1]} columns of data in this dataset")

There are 2611 rows of data in this dataset.
There are 9 columns of data in this dataset


In [191]:
# Check to ensure we have a complete dataset
fantasy_df.isna().sum()

title           0
selftext        0
subreddit       0
created_utc     0
author          0
num_comments    0
score           0
is_self         0
timestamp       0
dtype: int64

### The Scifi Subreddit
Similar to the Fantasy subreddit, we determined that the scifi subreddit reaches 100 posts every 4 days or so, and our goal was for a dataframe of about 2,500 rows. So, we collected 280 days of data, which meant that from the 7,000 original rows, about 4,500 rows were dropped due to being duplicates, nontext posts, or removed or deleted posts.

In [172]:
scifi_df = query_pushshift("scifi", day_window = 4, n = 70)

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=4d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=8d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=12d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=16d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=20d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=24d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=28d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=32d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=36d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=scifi&size=100&after=40d
Qu

In [193]:
# Take a look at the data
scifi_df.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,AI &amp; Emotions,"So lets talk AI, i know, how original. \n\nI h...",scifi,1595109322,VonBraun12,26,3,True,2020-07-18
1,Guess the book,"Hi all can you help, I’m trying to get the nam...",scifi,1595115628,DarthKittens,36,10,True,2020-07-18
2,I want to play a game. Describe your favorite ...,"I'll go first. ""Lonely man blogs about monocu...",scifi,1595117078,MyMomSaysIAmCool,83,7,True,2020-07-18
3,Zero Hours is an epic seven episode miniseries...,"What does it mean for the world to end? Okay, ...",scifi,1595119454,ArthurDrakoni,2,0,True,2020-07-18
4,Help needed - I’m writing a whodunit detective...,"So this is my idea, At the place of crime, the...",scifi,1595143575,gooodfella,10,0,True,2020-07-19


In [198]:
# Confirm the size of our data
print (f"There are {scifi_df.shape[0]} rows of data in this dataset.")
print (f"There are {scifi_df.shape[1]} columns of data in this dataset")

There are 2595 rows of data in this dataset.
There are 9 columns of data in this dataset


In [185]:
# Check that we have a completed dataset
scifi_df.isna().sum()

title           0
selftext        0
subreddit       0
created_utc     0
author          0
num_comments    0
score           0
is_self         0
timestamp       0
dtype: int64

### Combining the Dataframes
Next, we'll concatenate the 2 datasets, one on top of the other, into one large dataframe

In [186]:
reddit_df = pd.concat([fantasy_df, scifi_df], axis = 0, ignore_index= True)
# Reset the index so there are no repeats
reddit_df.reset_index(drop = True, inplace = True)
# Take a look at our data
reddit_df.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,What are NEWLY published and noticeably popula...,I've asked the same question last year I think...,Fantasy,1595292735,uera,68,37,True,2020-07-20
1,LF Recommendation: Fantasy with long travel ep...,Hi looking for recommendation for a top prefer...,Fantasy,1595295818,Overthrown77,16,5,True,2020-07-20
2,What is the best Fantasy book you've read that...,usually I don't enjoy something below four sta...,Fantasy,1595304562,OraclePreston,55,19,True,2020-07-21
3,Favorite fantasy names?,"I love fantasy, and I also have a thing for na...",Fantasy,1595305376,omnomenclature,56,17,True,2020-07-21
4,Anyone can recommend me a book without having ...,Idk how else to put the title but I really wan...,Fantasy,1595305716,UlyssesCourier,61,33,True,2020-07-21


In [199]:
# Confirm the size of our data
print (f"There are {reddit_df.shape[0]} rows of data in this dataset.")
print (f"There are {reddit_df.shape[1]} columns of data in this dataset")

There are 5206 rows of data in this dataset.
There are 9 columns of data in this dataset


In [187]:
# Check that our 2 dataframes are in our new one
reddit_df["subreddit"].value_counts()

Fantasy    2611
scifi      2595
Name: subreddit, dtype: int64

### Export our Dataframe

In [188]:
# Export our dataframe, while not creating a new column from the index.
reddit_df.to_csv("subreddits.csv", index = False)